In [ ]:
# загрузка модели -- для colab не обязательно

!apt-get install git-lfs
!git lfs install

#!git clone https://huggingface.co/spaces/Aniemore/Russian-Emotion-Recognition
!git clone https://huggingface.co/Aniemore/wav2vec2-xlsr-53-russian-emotion-recognition


!ls | awk '{system("rm -rf " $1 "/.git")}'
!ls -Ra

## Setups

In [ ]:
import torchaudio
torchaudio.__version__

In [1]:
%pip install torchaudio===0.11.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html
     -------------------------------------- 573.4/573.4 kB 7.2 MB/s eta 0:00:00
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.12.1
    Uninstalling torchaudio-0.12.1:
      Successfully uninstalled torchaudio-0.12.1
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip uninstall torchaudio

^C
Note: you may need to restart the kernel to use updated packages.


In [10]:
# https://huggingface.co/Aniemore/wav2vec2-xlsr-53-russian-emotion-recognition  -- адрес модели
# https://huggingface.co/spaces/Aniemore/Russian-Emotion-Recognition  -- работающая демо-версия
# https://huggingface.co/spaces/Aniemore/Russian-Emotion-Recognition/blob/main/app.py  -- код демо-версии на сайте


#%pip install numpy==1.21
#pip cache list
import numpy as np
np.__version__

# Работает:
#%pip install torch==1.11.0+cu113  -f https://download.pytorch.org/whl/cu113/torch_stable.html
#%pip install torchaudio===0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

#%pip install torch==1.9.0+cu113   -f https://download.pytorch.org/whl/cu113/torch_stable.html
#%pip install torchaudio==0.9.0+cu113   -f https://download.pytorch.org/whl/cu113/torch_stable.html
#%pip install transformers==4.8.0
# torchvision==0.10.0  torchtext==0.10.0 


# Не работает
#%pip install torch==1.11.0+cu113 torchaudio===0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
#%pip install torch==1.11.0 torchaudio===0.10.0 -f https://download.pytorch.org/whl/cu113/torch_stable.htm
#+cu113   torchvision==0.11.1+cu113
# --upgrade torch

import torch
print('torch   :', torch.__version__)

import torch.nn as nn
import torch.nn.functional as F
import torchaudio


#%pip install transformers --upgrade
import transformers
print('transformers', transformers.__version__)  
#  if you don’t see at least 4.3.0, you will need to upgrade your install
from transformers import AutoConfig, AutoModel, Wav2Vec2FeatureExtractor

import librosa



def speech_file_to_array_fn(path, sampling_rate):
    speech_array, _sampling_rate = torchaudio.load(path)
    resampler = torchaudio.transforms.Resample(_sampling_rate)
    speech = resampler(speech_array).squeeze().numpy()
    return speech


def predict(path, sampling_rate):
    speech = speech_file_to_array_fn(path, sampling_rate)
    inputs = feature_extractor(speech, sampling_rate=sampling_rate, return_tensors="pt", padding=True)
    inputs = {key: inputs[key].to(device) for key in inputs}

    with torch.no_grad():
        logits = model_(**inputs).logits

    scores = F.softmax(logits, dim=1).detach().cpu().numpy()[0]
    outputs = [{"Emotion": config.id2label[i], "Score": f"{round(score * 100, 3):.1f}%"} for i, score in enumerate(scores)]
    return outputs

torch   : 1.11.0+cu113
transformers 4.21.2


In [11]:
from transformers import AutoConfig, AutoModel, Wav2Vec2FeatureExtractor
import torch

TRUST = True

config = AutoConfig.from_pretrained('Aniemore/wav2vec2-xlsr-53-russian-emotion-recognition', trust_remote_code=TRUST)
model_ = AutoModel.from_pretrained("Aniemore/wav2vec2-xlsr-53-russian-emotion-recognition" , trust_remote_code=TRUST) # , trust_remote_code=TRUST
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("Aniemore/wav2vec2-xlsr-53-russian-emotion-recognition")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_.to(device)

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


Wav2Vec2ForSpeechClassification(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (1): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (2): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (3): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), 

### Эксперименты 

In [ ]:
import requests
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import Audio, display

In [ ]:
#song_url = 'https://drive.google.com/file/d/1NyJ9FhJyiKn9SZnepjyiiQwbPYSk5l-4/view?usp=sharing'
song_url = 'https://cloud.mail.ru/public/2TQz/1qCMFGVUG'

with requests.get(song_url, stream=True) as response:
  waveform, sample_rate = torchaudio.load(response.raw)
plot_specgram(waveform, sample_rate, title="HTTP datasource")

In [ ]:
# брать кусками torch?

torch.cuda.empty_cache()

#with torch.no_grad():
predict('0fIFjeeLJjIBliWUCsxyMF.wav', 16000) 
#predict('6XjpxthF6oeDDSYZX0CJq0.wav', 16000) 

# файл тут: https://drive.google.com/file/d/1NyJ9FhJyiKn9SZnepjyiiQwbPYSk5l-4/view?usp=sharing

# Такая же ошибка -- https://stackoverflow.com/questions/73433868/systemerror-google-protobuf-pyext-descriptor-cc358-bad-argument-to-internal


In [ ]:
short

(array([-1., -1., -1., ..., -1., -1., -1.], dtype=float32), 16000)

In [ ]:
# Расчет сокращения: сколько сейчас секунд*(свободно/занимает)
243*(2.23/9.2)

58.901086956521745

In [8]:
import numpy as np
np.__version__

'1.21.6'

In [12]:
# файлик на компе
sound_emo_pred = predict(f'1Mw433bxzwrR1boiIcovdZ.wav', 16000)

RuntimeError: Numpy is not available

## sound_emos

In [ ]:
# Полный микс песни (от 40-й до 100-й секунды)

import os
import pandas as pd

sound_emos = pd.DataFrame()

'''
for filename in os.listdir('./beg/'):  # [:2]
    sound_emo_pred = predict(f'./beg/{filename}', 16000)
    #print(sound_emo_pred)
    #display(pd.DataFrame.from_dict(sound_emo_pred).set_index('Emotion').T.style.background_gradient(cmap="Blues", axis=1)) # text_emo_pred[0]
    sound_emos = sound_emos.append(pd.DataFrame.from_dict(sound_emo_pred).set_index('Emotion').T)
    sound_emos['track_id'] = filename[:-4]
'''

for filename in os.listdir('./beg/'):  # [:2]
    sound_emo_pred = predict(f'./beg/{filename}', 16000)
    track_id = {'Emotion':'track_id', 'Score':filename[:-4]}
    sound_emos = sound_emos.append(pd.DataFrame.from_dict(sound_emo_pred).append(track_id, ignore_index=True).set_index('Emotion').T)
    #display(sound_emos)

sound_emos.reset_index(drop=True, inplace=True)
print(len(sound_emos), 'lines')
sound_emos.style.background_gradient(cmap="Blues", axis=1)


# Очищаем от "%"
for col in sound_emos.columns[:-1]:
    sound_emos[col] = pd.to_numeric(sound_emos[col].str.strip('%'))


# Сохраняем и скачиваем
sound_emos.to_csv('sound_emos.csv')

from google.colab import files 
files.download('sound_emos.csv')

In [ ]:
sound_emos

Emotion,anger,disgust,enthusiasm,fear,happiness,neutral,sadness,track_id
0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0Vqap6DWhvILPwGvbtlUi5
1,1.6,0.7,0.0,0.0,0.0,0.0,97.6,3U7fJ0VfFR1Msntjgd3DPB
2,0.0,0.0,0.0,0.0,0.0,0.0,100.0,6fOOKqQWcSy33ApU1wqkkW
3,0.0,0.0,0.0,0.0,0.0,0.0,100.0,7qveFKh99XSlkED6hTPAoX
4,0.0,0.0,0.0,0.0,0.0,0.0,100.0,3MiqKj1HQeOONdXLrpdOt8
...,...,...,...,...,...,...,...,...
295,0.7,58.1,0.0,0.4,1.1,0.1,39.5,3qhfXNcG8fGZdyNhiBpeDD
296,73.3,19.4,0.0,1.3,2.9,2.8,0.3,2MvsO4jP0zj8nBrKEfZaWt
297,0.0,0.0,0.0,0.0,0.0,0.0,100.0,71NeNcFTkxPPaGf1Er96Ss
298,0.0,99.9,0.0,0.0,0.0,0.0,0.0,6g1NlCpW7fgqDnWbCCDrHl


In [ ]:
# Голос

import os
import pandas as pd

sound_emos = pd.DataFrame()

'''
for filename in os.listdir('./beg/'):  # [:2]
    sound_emo_pred = predict(f'./beg/{filename}', 16000)
    #print(sound_emo_pred)
    #display(pd.DataFrame.from_dict(sound_emo_pred).set_index('Emotion').T.style.background_gradient(cmap="Blues", axis=1)) # text_emo_pred[0]
    sound_emos = sound_emos.append(pd.DataFrame.from_dict(sound_emo_pred).set_index('Emotion').T)
    sound_emos['track_id'] = filename[:-4]
'''

for filename in os.listdir('./voice/')[:]:  # [:2]
    sound_emo_pred = predict(f'./voice/{filename}', 16000)
    track_id = {'Emotion':'track_id', 'Score':filename[:-4]}
    sound_emos = sound_emos.append(pd.DataFrame.from_dict(sound_emo_pred).append(track_id, ignore_index=True).set_index('Emotion').T)
    #display(sound_emos)
    

sound_emos.reset_index(drop=True, inplace=True)
print(len(sound_emos), 'lines')
sound_emos.style.background_gradient(cmap="Blues", axis=1)


# Очищаем от "%"
for col in sound_emos.columns[:-1]:
    sound_emos[col] = pd.to_numeric(sound_emos[col].str.strip('%'))

    
# Сохраняем и скачиваем
sound_emos.to_csv('voice_emos.csv')

from google.colab import files 
files.download('voice_emos.csv')

300 lines


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
sound_emos

Emotion,anger,disgust,enthusiasm,fear,happiness,neutral,sadness,track_id
0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0Vqap6DWhvILPwGvbtlUi5
1,100.0,0.0,0.0,0.0,0.0,0.0,0.0,3U7fJ0VfFR1Msntjgd3DPB
2,7.7,89.0,0.0,0.2,0.0,0.0,3.0,6fOOKqQWcSy33ApU1wqkkW
3,21.0,0.1,0.0,0.4,77.9,0.0,0.5,7qveFKh99XSlkED6hTPAoX
4,0.0,99.6,0.0,0.0,0.0,0.0,0.3,3MiqKj1HQeOONdXLrpdOt8
...,...,...,...,...,...,...,...,...
295,2.1,72.0,0.0,0.8,0.4,0.0,24.7,3qhfXNcG8fGZdyNhiBpeDD
296,99.3,0.1,0.0,0.1,0.2,0.0,0.2,2MvsO4jP0zj8nBrKEfZaWt
297,0.0,0.1,0.0,0.0,0.0,0.0,99.9,71NeNcFTkxPPaGf1Er96Ss
298,99.8,0.1,0.0,0.0,0.0,0.0,0.0,6g1NlCpW7fgqDnWbCCDrHl


In [ ]:
# Сохраняем
len(sound_emos)

25

## Анализ

#### sound_emos 12

In [ ]:

import os
import pandas as pd

sound_emos = pd.DataFrame()

for filename in os.listdir('./voice/'):
    sound_emo_pred = predict(f'./voice/{filename}', 16000)
    #print(sound_emo_pred)
    #display(pd.DataFrame.from_dict(sound_emo_pred).set_index('Emotion').T.style.background_gradient(cmap="Blues", axis=1)) # text_emo_pred[0]
    sound_emos = sound_emos.append(pd.DataFrame.from_dict(sound_emo_pred).set_index('Emotion').T)

sound_emos.reset_index(drop=True, inplace=True)
print(len(sound_emos), 'lines')
sound_emos.style.background_gradient(cmap="Blues", axis=1)
# голос

12 lines


Emotion,anger,disgust,enthusiasm,fear,happiness,neutral,sadness
0,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,100.0%
1,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,100.0%
2,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,99.9%
3,94.7%,0.0%,0.0%,0.2%,0.0%,0.0%,4.9%
4,77.6%,21.0%,0.0%,0.3%,0.7%,0.0%,0.5%
5,0.1%,99.9%,0.0%,0.0%,0.0%,0.0%,0.0%
6,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,100.0%
7,0.0%,0.0%,0.0%,0.0%,0.0%,99.9%,0.0%
8,2.2%,0.0%,0.0%,0.1%,96.9%,0.0%,0.7%
9,1.9%,0.1%,0.0%,0.1%,38.3%,0.0%,59.6%


In [ ]:
# начало
sound_emos.style.background_gradient(cmap="Blues", axis=1)

Emotion,anger,disgust,enthusiasm,fear,happiness,neutral,sadness
0,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,100.0%
1,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,100.0%
2,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,100.0%
3,100.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%
4,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,100.0%
5,0.0%,100.0%,0.0%,0.0%,0.0%,0.0%,0.0%
6,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,100.0%
7,0.1%,0.0%,0.0%,0.0%,0.0%,99.9%,0.0%
8,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,100.0%
9,0.0%,0.0%,0.0%,0.0%,0.0%,100.0%,0.0%


In [ ]:
# конец
sound_emos.style.background_gradient(cmap="Blues", axis=1)

Emotion,anger,disgust,enthusiasm,fear,happiness,neutral,sadness
0,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,99.9%
1,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,99.9%
2,62.1%,16.7%,0.0%,0.6%,1.9%,0.1%,18.6%
3,100.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%
4,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,100.0%
5,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,100.0%
6,0.1%,0.1%,0.0%,0.0%,0.0%,0.0%,99.9%
7,0.0%,0.0%,0.0%,0.0%,0.0%,99.9%,0.0%
8,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,100.0%
9,99.9%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%


#### sound_emos 300

In [ ]:
sound_emos.mean(numeric_only=True)

Series([], dtype: float64)